In [1]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)

# imports
from common.utils import *

import warnings
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from common.custom_logistic_regression import CustomLogisticRegressionMulticlass
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
data = get_data()

X = data.drop("Target", axis=1)
y = data["Target"]

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), X.select_dtypes(include=["object"]).columns),
        ("num", StandardScaler(), X.select_dtypes(include=["int64", "float64"]).columns.tolist())
    ]
)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,single,2nd phase - general contingent,6th choice,Animation and Multimedia Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 3rd Cycle,Other - 11th Year,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,single,International student (bachelor),2nd choice,Tourism,Daytime,Secondary education,160.0,Portuguese,Secondary Education - 12th Year or Eq.,Higher Ed - Degree,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,single,1st phase - general contingent,6th choice,Communication Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 1st Cycle (4th/5th),...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,single,2nd phase - general contingent,3rd choice,Journalism and Communication,Daytime,Secondary education,122.0,Portuguese,Basic Ed 2nd Cycle (6th–8th),Basic Ed 1st Cycle (4th/5th),...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,married,Over 23 years old,2nd choice,Social Service (evening attendance),Evening,Secondary education,100.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 2nd Cycle (6th–8th),...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [3]:
get_custom_dataframe(LogisticRegression(max_iter=250), preprocessor, accuracy_score, X_train_val,
                        y_train_val, X_test, y_test)

,0,1,2,3,4
Train,0.819852,0.812434,0.820911,0.824444,0.822324
Validation,0.790960,0.802260,0.788136,0.752825,0.772599
Test,0.757062,0.768362,0.766102,0.764972,0.768362


In [4]:
warnings.filterwarnings("ignore", category=FutureWarning)

model = CustomLogisticRegressionMulticlass(epochs=600, batch_size=256, learning_rate=0.01)
result = get_custom_dataframe(model, preprocessor, accuracy_score, X_train_val,
                        y_train_val, X_test, y_test)

result

,0,1,2,3,4
Train,0.783822,0.783822,0.783116,0.781703,0.782056
Validation,0.765537,0.786723,0.765537,0.779661,0.795198
Test,0.753672,0.755932,0.751412,0.754802,0.757062


In [5]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=200))
])

pipeline.fit(X_train_val, y_train_val)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.75      0.77       295
           1       0.52      0.36      0.43       140
           2       0.81      0.90      0.85       450

    accuracy                           0.77       885
   macro avg       0.70      0.67      0.68       885
weighted avg       0.75      0.77      0.76       885



In [6]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", CustomLogisticRegressionMulticlass(epochs=600, batch_size=256, learning_rate=0.01))
])

pipeline.fit(X_train_val, y_train_val)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.77      0.78       295
           1       0.43      0.23      0.30       140
           2       0.79      0.92      0.85       450

    accuracy                           0.76       885
   macro avg       0.67      0.64      0.64       885
weighted avg       0.73      0.76      0.74       885

